In [1]:
import mydemosaicking_header as mh

#read data
train_gt_path = 'E:/data/DIV2K/DIV2K_train_patch/train/gt/'
train_train_path = 'E:/data/DIV2K/DIV2K_train_patch/train/train/'
valid_gt_path = 'E:/data/DIV2K/DIV2K_train_patch/valid/gt/'
valid_valid_path = 'E:/data/DIV2K/DIV2K_train_patch/valid/valid/'
train_gt_patch = []
train_train_patch = []
valid_gt_img = []
valid_valid_img = []
rgb = ['r','g','b']

for i in rgb:
    gt_patch = '_gt_patch_all.h5'
    train_patch = '_train_patch_all.h5'
    gt_filename = i+gt_patch
    train_filename = i+train_patch
    temp = mh.load_h5(train_gt_path, gt_filename, 1)
    train_gt_patch.append(temp)
    temp = mh.load_h5(train_train_path, train_filename, 1)
    train_train_patch.append(temp)
    for j in range(1,101):
        valid_img = '_valid_img_'
        gt_img = '_gt_img_'
        h5 = '.h5'
        valid_filename = str(j) + valid_img + i + h5
        gt_filename = str(j) + gt_img + i + h5
        temp = mh.load_h5(valid_valid_path, valid_filename, j)
        valid_valid_img.append(temp)
        temp = mh.load_h5(valid_gt_path, gt_filename, j)
        valid_gt_img.append(temp)

Using TensorFlow backend.
